# Описание проекта
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

### Постановка задачи
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных
Данные находятся в файле `toxic_comments.csv`. 
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### [1. Подготовка](#1-bullet)
### [2. Обучение](#2-bullet)
### [3. Выводы](#3-bullet)

#### Библиотеки

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import spacy
from tqdm import notebook

# 1. Подготовка <a id='1-bullet'></a>

In [2]:
df = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/12. Модель классификации текста на положительный и отрицательный/Рабочие файлы/toxic_comments.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
# Проверим баланс классов
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [4]:
# Разделим наши данные на признаки и целевой признак
features = df.drop(['toxic'], axis = 1)
target = df['toxic']

In [5]:
# Удалим 50% значений с 0 целевым признаком, так как их слишком много по отношению к 1 
from sklearn.utils import shuffle # импортируем функцию перемешивания сообщений
def downsample(features, target, fraction):
    features_zeros = features[target == 0].sample(frac=fraction, random_state=12345)
    features_ones = features[target == 1]
    target_zeros = target[target == 0].sample(frac=fraction, random_state=12345)
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones])
    target_upsampled = pd.concat([target_zeros] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [6]:
features_down, target_down = downsample(features, target, 0.5)

In [7]:
features_down.shape

(87898, 1)

In [8]:
# Разобъем наши данные на тренировочную и тестовую выборки:  70/30%
features_train, features_test, target_train, target_test = (
                train_test_split(features_down, target_down, test_size=0.3))

In [9]:
# Загрузим модель английского языка en Spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
# Загрузим стопслова английского языка en
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [10]:
# Токенизация текста
def spacy_tokenize(text):
    doc = nlp.tokenizer(text)
    return [token.text for token in doc]

In [12]:
# Удаление стопслов из текста
def remove_stopwords(tokens):
    cleaned_tokens = []
    for token in tokens:
        if token not in spacy_stopwords:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [13]:
# Лемматизация текста с предварительной токенизацией
def spacy_lemmatize(text):
    doc = nlp.tokenizer(text)
    return [token.lemma_ for token in doc]

In [14]:
def spacy_lemm(text):
    doc = nlp.tokenizer(text.lower()) # токенизируем текст, предварительно переведя в нижний регистр
    lemm_text = [token.lemma_ for token in doc] # лемматизируем текст
    
    cleaned_tokens = []
    for token in lemm_text: # Удалим стопслова из текста
        if token not in spacy_stopwords:
            cleaned_tokens.append(token)
        
    return cleaned_tokens

In [15]:
#spacy_lemm(df['text'].iloc[0])

In [16]:
#list(df['text'][:1])

In [17]:
# Функция по обработке сообщений всего датасета
def lemm_array(df):
    array = []
    for i in notebook.tqdm(range(df.shape[0])):
        message = " ".join(spacy_lemm(df.iloc[i])) # Соберем сообщение в один текст
        message_re = re.sub(r'[^a-zA-Z]', ' ', message) # Удалим из сообщения посторонние символы и цифры
        list_j = " ".join(message_re.split()) # Удалим лишние пробелы
        array.append(list_j) # Соберем все сообщения в единый список
    return array

In [18]:
lemm_array(df['text'][:2]) # Проверим качество проделанной работы

  0%|          | 0/2 [00:00<?, ?it/s]

['explanation edit username hardcore metallica fan revert vandalisms closure gas vote new york doll fac remove template talk page PRON retire now',
 'd aww match background colour PRON seemingly stick thank talk january utc']

In [19]:
print(list(df['text'][:2])) # Для сравнения

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27", "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"]


In [20]:
del df

In [21]:
# Корпус с тренировочной выборкой
corpus_train = lemm_array(features_train['text'])

  0%|          | 0/61528 [00:00<?, ?it/s]

In [22]:
# Корпус с тестовой выборкой
corpus_test = lemm_array(features_test['text'])

  0%|          | 0/26370 [00:00<?, ?it/s]

In [23]:
# Подготовим модель по обращению подготовленного текста в матрицу с удалением стопслов
tf_idf = TfidfVectorizer().fit(corpus_train)

In [24]:
# Тренировочная матрица
features_tr = tf_idf.transform(corpus_train)
# Тестовая матрица
features_tt = tf_idf.transform(corpus_test)
# Удалим неиспользуемые в дальнейшем данные
del corpus_train
del corpus_test

In [25]:
# Проверим соответствие тренировочной матрицы признаков и целевого признака
print(target_train.shape)
features_tr.shape

(61528,)


(61528, 84269)

In [26]:
# Проверим соответствие тестовой матрицы признаков и целевого признака
print(target_test.shape)
features_tt.shape

(26370,)


(26370, 84269)

### Вывод
1. Исходный файл состоит из 159571 сообщений с размеченой областью "toxic";
2. Текст содержит много слэнговых выражений, цифр, символов и часто состоит не из одного предложения, что осложняет адекватный его перевод в цифровой вид пригодный для обучения модели;
3. Мы изначально разделили данные на обучающую и тестовую выборки для удобства обработки;
4. Подготовка текста переводу в матричный вид состоит из двух основных этапов. На первом этапе мы создали функцию lemmatize которая способна комплексно обработать одно сообщение: удалить регулярные выражения, пробелы и провести лемматизацию. На втором этапе мы создали функцию lemm_array с помошью которой по порядку скармливаем наши сообщения для комплексной обработки.
5. Итог нашей подготовки 2 матрицы признаков - обучающая и тестовая.

# 2. Обучение<a id='2-bullet'></a>

In [27]:
# Найдем лучшие гиперпараметры LogisticRegression с помощью GridSearchCV
#, для балансировки целевого признака применим class_weight='balanced'
LR_params = {'C': range(9, 15), 'solver': ['sag', 'liblinear']}

In [28]:
%%time
model_LR = LogisticRegression(random_state=287, class_weight='balanced')
LR_class_weight = GridSearchCV(model_LR, LR_params, cv=3, n_jobs=-1, scoring='f1', verbose=True)
# Обучим модель
LR_class_weight.fit(features_tr, target_train)
print(LR_class_weight.best_params_, LR_class_weight.best_score_) 

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   30.8s finished


{'C': 9, 'solver': 'sag'} 0.8091259925401583
Wall time: 32.8 s


In [29]:
#Модель логистической регрессии, для балансировки целевого признака применим class_weight='balanced'
model_logistic = LogisticRegression(random_state=287, class_weight='balanced', solver='liblinear', C=9)
model_logistic.fit(features_tr, target_train) # обучим модель на тренировочной выборке
predict_m = model_logistic.predict(features_tt)
print('Скоринг модели Логистической регрессии F1:',round(f1_score(target_test, predict_m), 2))

Скоринг модели Логистической регрессии F1: 0.83


In [30]:
# Найдем лучшие гиперпараметры LGBMClassifier с помощью GridSearchCV
LGBM_params = {'max_depth': range(2,4),'num_leaves': range(2,4),
               'n_estimators': range(400, 700, 100)}

In [31]:
%%time
# Найдем оптимальные гиперпараметры, для балансировки целевого признака применим class_weight='balanced'
model_LGBM = LGBMClassifier(random_state=287, class_weight='balanced')

LGBM_class_weight = GridSearchCV(model_LGBM, LGBM_params, cv=3, n_jobs=-1, 
                                 scoring='f1', verbose=True)
# Обучим модель
LGBM_class_weight.fit(features_tr, target_train)
print(LGBM_class_weight.best_params_, LGBM_class_weight.best_score_) 

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.0min finished


{'max_depth': 2, 'n_estimators': 600, 'num_leaves': 3} 0.7940684386978575
Wall time: 3min 15s


In [32]:
#Модель LGBM, для балансировки целевого признака применим class_weight='balanced'
model_LGBM = LGBMClassifier(random_state=287, 
                            class_weight='balanced', 
                            num_leaves=3,
                            max_depth=2, n_estimators=600)
model_LGBM.fit(features_tr, target_train) # обучим модель на тренировочной выборке
predict_m = model_LGBM.predict(features_tt)
print('Скоринг модели LGBMClassifier F1:',round(f1_score(target_test, predict_m), 2))

Скоринг модели LGBMClassifier F1: 0.8


# 3. Выводы<a id='3-bullet'></a>
1. Для поиска лучших гиперпараметров для наших моделей LogisticRegression и LGBMClassifier мы применили GridSearchCV;
2. В виду того что целевой признак имеет сильный дисбаланс в отрицательную сторону при обучении моделей мы применили class_weight='balanced';
3. Лучшей из двух моделей по итогам испытания оказалась Логистическая регрессия с значением скоринга F1 = 0.83, что достаточно по условиям поставленной задачи.